In [2]:
import pygeoprocessing as pg
import numpy as np
from osgeo import gdal

In [ ]:
# WARP STAKEHOLDER RASTER TO MATCH PADDED SEALS RASTER
# Input file paths
raster_path = "/Users/mbraaksma/Library/CloudStorage/GoogleDrive-braak014@umn.edu/Shared drives/NatCapTEEMs/Projects/JRC Project/Data/stakeholder/Raster files/RS1_Landcover/Land Use Land Cover-2020-2021-2022-2023/Land Use Land Cover/2020_composite_raw.tif"
aligned_raster_path = "/Users/mbraaksma/Files/base_data/demetra-seals/lulc/stakeholder2seals/2020_composite_10sec.tif"

# Reference raster for target properties
ref_raster = "/Users/mbraaksma/Files/seals/projects/run_ken_standard/intermediate/fine_processed_inputs/lulc/esa/seals7/lulc_esa_seals7_2017.tif"
ref_raster_info = pg.get_raster_info(ref_raster)

# Warp raster with exact settings
pg.warp_raster(
    base_raster_path=raster_path,
    target_pixel_size=ref_raster_info['pixel_size'],  # Match pixel size
    target_raster_path=aligned_raster_path,
    resample_method="near",  # Nearest neighbor for categorical data
    target_bb=ref_raster_info["bounding_box"],  # Exact bounding box
    gdal_warp_options=["-tap"],  # Align pixels to avoid rounding issues
)

In [ ]:
# CONVERT OUT OF BOUNDS VALUES TO NODATA
# Input file paths
raster_path = "/Users/mbraaksma/Files/base_data/demetra-seals/lulc/stakeholder2seals/2020_composite_10sec.tif"
aez_raster_path = '/Users/mbraaksma/Files/base_data/demetra-seals/borders/kenya_aez_300m.tif'

# Load rasters
raster_array = pg.raster_to_numpy_array(raster_path)
aez_array = pg.raster_to_numpy_array(aez_raster_path)
unique_values = np.unique(raster_array)
print("Unique values in original raster:", unique_values) 
print("Unique values in AEZ raster:", np.unique(aez_array))

# Reclassify out of bounds values to nodata
raster_array[aez_array == 255] = 255
print("Unique values in reclassified raster:", np.unique(raster_array))

# Save reclassified raster
ref_raster_info = pg.get_raster_info(aez_raster_path)
fixed_raster_path = "/Users/mbraaksma/Files/base_data/demetra-seals/lulc/stakeholder2seals/2020_composite_10sec_fixedNDV.tif"
pg.numpy_array_to_raster(raster_array, 
                        target_nodata=255, 
                        pixel_size=ref_raster_info['pixel_size'], 
                        origin=[ref_raster_info['bounding_box'][0], ref_raster_info['bounding_box'][-1]],
                        projection_wkt=ref_raster_info['projection_wkt'], 
                        target_path=fixed_raster_path)

Unique values in original raster: [0 1 2 3 4 5 6 7 8]
Unique values in AEZ raster: [  1   2   3   4   5   6   7   8 255]
Unique values in reclassified raster: [  0   1   2   3   4   5   6   7   8 255]


In [ ]:
# RECLASSIFY STAKEHOLDER RASTER AND CONVERT OUTOFBOUNDS VALUES TO NODATA
# Input file paths
raster_path = fixed_raster_path
reclassified_raster_path = "/Users/mbraaksma/Files/base_data/demetra-seals/lulc/stakeholder2seals/2020_composite_seals7.tif"
aez_raster_path = '/Users/mbraaksma/Files/base_data/demetra-seals/borders/kenya_aez_300m.tif'

reclass_map = {
    5: 5,  # Shrub and Scrub -> Othernat
    3: 5,  # Flooded Vegetation -> Othernat
    7: 7,  # Bare -> Other
    6: 1,  # Built -> Urban
    4: 2,  # Crops -> Cropland
    2: 3,  # Grass -> Grassland
    1: 4,  # Trees -> Forest
    8: 7,  # Snow and Ice -> Other
    0: 6   # Water and nodata -> Water
}

# recalssify raster
pg.reclassify_raster(
    base_raster_path_band=(raster_path,1),
    value_map=reclass_map,
    target_raster_path=reclassified_raster_path,
    target_datatype=3,
    target_nodata=255,
    values_required=True,
)
